In [715]:
import warnings
warnings.filterwarnings("ignore")

import numpy
import numpy as np
import pandas as pd
import itertools
import os
import csv
import re
import nltk
from nltk.tokenize import sent_tokenize

# Import adjustText, initialize list of texts
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA



from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score,fbeta_score

from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm


import seaborn as sns
import matplotlib.pyplot as plt



from IPython.display import SVG


#Seed Random Numbers with the TensorFlow Backend
from numpy.random import seed
seed(1)

In [716]:
df_train = pd.read_excel('Training Dataset.xlsx', usecols=['Tweets', 'S/G'])
df_train.shape, df_train.columns 

((3564, 2), Index(['Tweets', 'S/G'], dtype='object'))

In [717]:
print("Total Training Dataset:" + str(len(df_train)))
print("Group class:" + str(len(df_train[(df_train['S/G'] == 0)])))
print("Individual class:" + str(len(df_train[(df_train['S/G'] == 1)])))

Total Training Dataset:3564
Group class:1341
Individual class:441


In [718]:
#remove all the rows having "2" in the G/S column

df_train = df_train[(df_train['S/G'] != 2)]
df_train = df_train.reindex()
df_train.head()

,Tweets,S/G
0,بکواس مت کرو,1
1,تمہاری ہیجڑا فورس ایک نہتے کے سامنے بکری بنی ہ...,0
2,آفیسر سمیتبھارتی فوجی جہنم واصل،بنکرز تباہ بھا...,1
3,غدار منافق میر اللہ تمہیں زلیل کرے,1
4,اگست پر آپ بھارت کو کیا پیغام دینا چاہیں گے؟ م...,0


In [719]:
df_test = pd.read_excel('data_test.xlsx', usecols=['Tweet', 'G/S'])
df_test.head()

,Tweet,G/S
0,وزیراعل ی پنجاب کا ضلع راولپنڈی کا دورہ لینڈسل...,2
1,الحمدلله پورے پاکستان کے وکلاء برادری کا کہنا ...,1
2,جنوری مہلت ختم الٹی گنتی شروع ن لیگ کے خلا...,1
3,کروناکی آڑ میں حکومت اورفوج جوکھیل قادیانیوں ک...,1
4,حکومت ایکشن میں آئے ورنہ اگر ہم ایکشن میں آگئے...,1


In [720]:
#remove all the rows having "2" in the G/S column

df_test = df_test[(df_test['G/S'] != 2)]
df_test = df_test.reindex()
df_test.head()

,Tweet,G/S
1,الحمدلله پورے پاکستان کے وکلاء برادری کا کہنا ...,1
2,جنوری مہلت ختم الٹی گنتی شروع ن لیگ کے خلا...,1
3,کروناکی آڑ میں حکومت اورفوج جوکھیل قادیانیوں ک...,1
4,حکومت ایکشن میں آئے ورنہ اگر ہم ایکشن میں آگئے...,1
8,زلفی بخاری قوم آپ سے حساب لے گی,1


## Baseline Model

In [721]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [722]:
def extract_ngrams(data, num=1):
    n_grams = nltk.ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]

In [723]:
df_train['unigrams'] = df_train['Tweets'].apply(extract_ngrams)
df_train['bigrams'] = df_train['Tweets'].apply(lambda x: extract_ngrams(x, 2))

df_test['unigrams'] = df_test['Tweet'].apply(extract_ngrams)
df_test['bigrams'] = df_test['Tweet'].apply(lambda x: extract_ngrams(x, 2))

## Vocabulary Set-Up

In [724]:
from collections import Counter

In [725]:
uni = Counter()
for idx, row in df_train.iterrows(): 
    uni.update(row['unigrams'])

In [726]:
df_vocab = pd.DataFrame.from_dict(uni, orient='index', columns=['count'])
df_vocab.reset_index(inplace=True)
df_vocab.columns = ['uniram', 'count']

In [727]:
df_vocab.describe()

,count
count,5220.000000
mean,5.847318
std,28.977964
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,814.000000


In [728]:
df_vocab['count'].sum()

30523

In [729]:
df_vocab.shape, df_vocab[df_vocab['count'] >= 3].shape

((5220, 2), (1427, 2))

In [730]:
vocab_uni = set(df_vocab[df_vocab['count'] >= 3]['uniram'])
len(vocab_uni)

1427

In [731]:
bi = Counter()
for idx, row in df_train.iterrows(): 
    bi.update(row['bigrams'])    
len(bi)   

19463

In [732]:
vocab_bi_df = pd.DataFrame.from_dict(bi, orient='index', columns=['count'])
vocab_bi_df.reset_index(inplace=True)
vocab_bi_df.columns = ['bigram', 'count']

In [733]:
vocab_bi_df.describe(percentiles=[.25, .5, .75, .8, .85, .9, .95, .99])

,count
count,19463.000000
mean,1.476699
std,2.041437
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
80%,1.000000
85%,2.000000
90%,2.000000


In [734]:
vocab_bi_df.shape, vocab_bi_df[vocab_bi_df['count'] >= 4].shape

((19463, 2), (922, 2))

In [735]:
vocab_bi = set(vocab_bi_df[vocab_bi_df['count'] >= 4]['bigram'])
len(vocab_bi)

922

In [736]:
vocab = vocab_uni.union(vocab_bi)
len(vocab)

2349

## Feature Engineering

In [737]:
import numpy as np

In [738]:
vocab_uni_li = list(vocab_uni)
vocab_bi_li = list(vocab_bi)

FEATURES = vocab_uni_li+vocab_bi_li + ['OOV_uni', 'OOV_bi']
len(vocab_uni_li) + len(vocab_bi_li), len(FEATURES)


(2349, 2351)

In [739]:
df_train.shape

(1782, 4)

In [740]:
stuffing = np.zeros((1782, 2351))
stuffing_df = pd.DataFrame(stuffing, columns = FEATURES)

In [741]:
df_train.head()

,Tweets,S/G,unigrams,bigrams
0,بکواس مت کرو,1,"[بکواس, مت, کرو]","[بکواس مت, مت کرو]"
1,تمہاری ہیجڑا فورس ایک نہتے کے سامنے بکری بنی ہ...,0,"[تمہاری, ہیجڑا, فورس, ایک, نہتے, کے, سامنے, بک...","[تمہاری ہیجڑا, ہیجڑا فورس, فورس ایک, ایک نہتے,..."
2,آفیسر سمیتبھارتی فوجی جہنم واصل،بنکرز تباہ بھا...,1,"[آفیسر, سمیتبھارتی, فوجی, جہنم, واصل،بنکرز, تب...","[آفیسر سمیتبھارتی, سمیتبھارتی فوجی, فوجی جہنم,..."
3,غدار منافق میر اللہ تمہیں زلیل کرے,1,"[غدار, منافق, میر, اللہ, تمہیں, زلیل, کرے]","[غدار منافق, منافق میر, میر اللہ, اللہ تمہیں, ..."
4,اگست پر آپ بھارت کو کیا پیغام دینا چاہیں گے؟ م...,0,"[اگست, پر, آپ, بھارت, کو, کیا, پیغام, دینا, چا...","[اگست پر, پر آپ, آپ بھارت, بھارت کو, کو کیا, ک..."


In [742]:
train_df = pd.concat([df_train, stuffing_df], axis=1)
train_df.shape

(1782, 2355)

In [743]:
def count_ngram_feats(df):    
    global vocab
    
    total_oov_uni = 0
    total_oov_bi = 0
    
    for idx, row in df.iterrows(): 
        # unigram part 
        ct = Counter()
        ct.update(row['unigrams'])
        oov_ct = 0 
        
        for k in ct.elements(): 
            if k in vocab:
                df[k].iloc[idx] = ct.get(k)
            else:     
                oov_ct += ct.get(k)
        df['OOV_uni'].iloc[idx] = oov_ct
        total_oov_uni += oov_ct

        # bigram part 
        ct = Counter()
        ct.update(row['bigrams'])
        oov_ct = 0 
        
        for k in ct.elements(): 
            if k in vocab:
                df[k].iloc[idx] = ct.get(k)
            else:     
                oov_ct += ct.get(k)
        df['OOV_bi'].iloc[idx] = oov_ct 
        total_oov_bi += oov_ct

    print (total_oov_uni, total_oov_bi)

In [744]:
count_ngram_feats(train_df)

4555 21712


In [745]:
train_df[list(vocab)[:3]].sum()

درندے         3.0
چین          10.0
پروپیگنڈہ     3.0
dtype: float64

In [746]:
train_df[['OOV_uni', 'OOV_bi']].sum()

OOV_uni     4555.0
OOV_bi     21712.0
dtype: float64

## Test Part

In [747]:
df_test.shape

(307, 4)

In [748]:
stuffing = np.zeros((307, 2351))
stuffing_df = pd.DataFrame(stuffing, columns=FEATURES)

In [749]:
stuffing_df

,چین,پروپیگنڈہ,چاہتی,گستاخ,امید,مردہ,واہ,پورا,ہوا,گے۔,...,نشان بنا,پر لعنت,گانڈ ماری,کا بچہ,نہیں ہے,یہ سب,سے دیکھا,بھی کر,OOV_uni,OOV_bi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [750]:
stuffing_df.index

RangeIndex(start=0, stop=307, step=1)

In [751]:
df_test.index

Int64Index([  1,   2,   3,   4,   8,  10,  11,  13,  14,  23,
            ...
            905, 909, 910, 915, 917, 919, 921, 927, 930, 931],
           dtype='int64', length=307)

In [752]:
df_test = df_test.reindex(index=stuffing_df.index)

In [753]:
test_df = pd.concat([df_test, stuffing_df], axis=1)
test_df.shape

(307, 2355)

In [754]:
test_df 

,Tweet,G/S,unigrams,bigrams,چین,پروپیگنڈہ,چاہتی,گستاخ,امید,مردہ,...,نشان بنا,پر لعنت,گانڈ ماری,کا بچہ,نہیں ہے,یہ سب,سے دیکھا,بھی کر,OOV_uni,OOV_bi
0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,الحمدلله پورے پاکستان کے وکلاء برادری کا کہنا ...,1.0,"[الحمدلله, پورے, پاکستان, کے, وکلاء, برادری, ک...","[الحمدلله پورے, پورے پاکستان, پاکستان کے, کے و...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,جنوری مہلت ختم الٹی گنتی شروع ن لیگ کے خلا...,1.0,"[جنوری, مہلت, ختم, الٹی, گنتی, شروع, ن, لیگ, ک...","[جنوری مہلت, مہلت ختم, ختم الٹی, الٹی گنتی, گن...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,کروناکی آڑ میں حکومت اورفوج جوکھیل قادیانیوں ک...,1.0,"[کروناکی, آڑ, میں, حکومت, اورفوج, جوکھیل, قادی...","[کروناکی آڑ, آڑ میں, میں حکومت, حکومت اورفوج, ...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,حکومت ایکشن میں آئے ورنہ اگر ہم ایکشن میں آگئے...,1.0,"[حکومت, ایکشن, میں, آئے, ورنہ, اگر, ہم, ایکشن,...","[حکومت ایکشن, ایکشن میں, میں آئے, آئے ورنہ, ور...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
303,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
304,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
305,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [755]:
# Repalce NaN with zero on all columns 
test_df = test_df.fillna(0)

## Model

In [756]:
!pip freeze

absl-py==1.3.0
aeppl==0.0.33
aesara==2.7.9
aiohttp==3.8.3
aiosignal==1.3.1
alabaster==0.7.12
albumentations==1.2.1
altair==4.2.0
appdirs==1.4.4
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.2
asynctest==0.13.0
atari-py==0.2.9
atomicwrites==1.4.1
attrs==22.1.0
audioread==3.0.0
autograd==1.5
Babel==2.11.0
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.1
blis==0.7.9
bokeh==2.3.3
branca==0.6.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==5.2.0
catalogue==2.0.8
certifi==2022.9.24
cffi==1.15.1
cftime==1.6.2
chardet==3.0.4
charset-normalizer==2.1.1
click==7.1.2
clikit==0.6.2
cloudpickle==1.5.0
cmake==3.22.6
cmdstanpy==1.0.8
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.3
cons==0.4.5
contextlib2==0.5.5
convertdate==2.4.0
crashtest==0.3.1
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.2.2
cycler==0.11.0
cymem==2.0.7
Cython==0.29.32
daft==0.0.4
dask==2022.2.0
datascience==0.17.5
db-dtypes==1.0.4
debugpy==1.0.

In [757]:
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import GradientBoostingClassifier 
#from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report, f1_score, roc_auc_score, accuracy_score
#from sklearn.model_selection import GridSearchCV

## Logistic Regression

In [758]:
logreg = LogisticRegression(max_iter=1000, class_weight='balanced')

In [759]:
logreg.fit(train_df[FEATURES], train_df['S/G'])

LogisticRegression(class_weight='balanced', max_iter=1000)

In [760]:
y = train_df['S/G']
y.shape

(1782,)

In [761]:
y_pred = logreg.predict(train_df[FEATURES])
y_pred_score = logreg.predict_proba(train_df[FEATURES])

In [762]:
f1_score(y, y_pred), roc_auc_score(y, y_pred_score[:, 1]), accuracy_score(y, y_pred)

(0.9381663113006397, 0.998254086620977, 0.9674523007856342)

In [763]:
y_pred_test = logreg.predict(test_df[FEATURES])
y_pred_score_test = logreg.predict_proba(test_df[FEATURES])
y_test = test_df['G/S']
f1_score(y_test, y_pred_test), roc_auc_score(y_test, y_pred_score_test[:, 1]), accuracy_score(y_test, y_pred_test)

(0.42966751918158563, 0.5, 0.2736156351791531)

## SVM

In [764]:
svc = svm.SVC(probability=True, class_weight='balanced')
svc.fit(train_df[FEATURES], train_df['S/G'])

SVC(class_weight='balanced', probability=True)

In [765]:
y_pred_svc = svc.predict(train_df[FEATURES])
y_pred_score_svc = svc.predict_proba(train_df[FEATURES])

In [766]:
f1_score(y, y_pred_svc), roc_auc_score(y, y_pred_score_svc[:, 1]), accuracy_score(y, y_pred_svc)

(0.589853826311264, 0.8588397327611134, 0.7323232323232324)

In [767]:
y_pred_test_svc = svc.predict(test_df[FEATURES])
y_pred_score_test_svc = svc.predict_proba(test_df[FEATURES])
y_test = test_df['G/S']
f1_score(y_test, y_pred_test_svc), roc_auc_score(y_test, y_pred_score_test_svc[:, 1]), accuracy_score(y_test, y_pred_test_svc)

(0.42966751918158563, 0.5, 0.2736156351791531)

## Random ForestClassifier

In [768]:
rf = RandomForestClassifier(max_features=None)
rf.fit(train_df[FEATURES], train_df['S/G'])

RandomForestClassifier(max_features=None)

In [769]:
y_pred = rf.predict(train_df[FEATURES])
y_pred_score = rf.predict_proba(train_df[FEATURES])

In [770]:
f1_score(y, y_pred), roc_auc_score(y, y_pred_score[:, 1]), accuracy_score(y, y_pred)

(0.9988649262202043, 0.9999974635640982, 0.999438832772166)

In [771]:
y_pred_test_rf = rf.predict(test_df[FEATURES])
y_pred_score_test_rf = rf.predict_proba(test_df[FEATURES])
y_test_rf = test_df['G/S']
f1_score(y_test_rf , y_pred_test_rf), roc_auc_score(y_test_rf, y_pred_score_test_rf[:, 1]), accuracy_score(y_test_rf, y_pred_test_rf)

(0.42966751918158563, 0.5, 0.2736156351791531)

## Gradient Boosting Classifier

In [772]:
gb = GradientBoostingClassifier()
gb.fit(train_df[FEATURES], train_df['S/G'])

GradientBoostingClassifier()

In [773]:
y_pred_gb = gb.predict(train_df[FEATURES])
y_pred_score_gb = gb.predict_proba(train_df[FEATURES])

In [774]:
f1_score(y, y_pred_gb), roc_auc_score(y, y_pred_score_gb[:, 1]), accuracy_score(y, y_pred_gb)

(0.55, 0.9372189840390542, 0.8383838383838383)

In [775]:
y_pred_test_gb  = gb.predict(test_df[FEATURES])
y_pred_score_test_gb = gb.predict_proba(test_df[FEATURES])
y_test_gb = test_df['G/S']
f1_score(y_test_gb , y_pred_test_gb), roc_auc_score(y_test_gb, y_pred_score_test_gb[:, 1]), accuracy_score(y_test_gb, y_pred_test_gb)

(0.42966751918158563, 0.5, 0.2736156351791531)

## AdaBoost Classifier

In [776]:
ad = AdaBoostClassifier()
ad.fit(train_df[FEATURES], train_df['S/G'])

AdaBoostClassifier()

In [777]:
y_pred_ad = ad.predict(train_df[FEATURES])
y_pred_score_ad = ad.predict_proba(train_df[FEATURES])

In [778]:
f1_score(y, y_pred_ad), roc_auc_score(y, y_pred_score_ad[:, 1]), accuracy_score(y, y_pred_ad)

(0.6285046728971962, 0.8800874563098916, 0.8215488215488216)

In [779]:
y_pred_test_ad  = ad.predict(test_df[FEATURES])
y_pred_score_test_ad = ad.predict_proba(test_df[FEATURES])
y_test_ad = test_df['G/S']
f1_score(y_test_ad , y_pred_test_ad), roc_auc_score(y_test_ad, y_pred_score_test_ad[:, 1]), accuracy_score(y_test_ad, y_pred_test_ad)

(0.42966751918158563, 0.5, 0.2736156351791531)

## GaussianNB

In [780]:
nb = GaussianNB()
nb.fit(train_df[FEATURES], train_df['S/G'])

GaussianNB()

In [781]:
y_pred_nb = nb.predict(train_df[FEATURES])
y_pred_score_nb = nb.predict_proba(train_df[FEATURES])

In [782]:
f1_score(y, y_pred_nb), roc_auc_score(y, y_pred_score_nb[:, 1]), accuracy_score(y, y_pred_nb)

(0.8724035608308605, 0.9519015659955257, 0.9276094276094277)

In [783]:
y_pred_test_nb  = nb.predict(test_df[FEATURES])
y_pred_score_test_nb = nb.predict_proba(test_df[FEATURES])
y_test_nb = test_df['G/S']
f1_score(y_test_nb , y_pred_test_nb), roc_auc_score(y_test_nb, y_pred_score_test_nb[:, 1]), accuracy_score(y_test_nb, y_pred_test_nb)

(0.42966751918158563, 0.5, 0.2736156351791531)

## Decision Tree Classifier

In [784]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(train_df[FEATURES], train_df['S/G'])

DecisionTreeClassifier(random_state=0)

In [785]:
y_pred_dt = dt.predict(train_df[FEATURES])
y_pred_score_dt = dt.predict_proba(train_df[FEATURES])

In [786]:
f1_score(y, y_pred_dt), roc_auc_score(y, y_pred_score_dt[:, 1]), accuracy_score(y, y_pred_dt)

(0.9988649262202043, 0.9999991545213661, 0.999438832772166)

In [787]:
y_pred_test_dt  = dt.predict(test_df[FEATURES])
y_pred_score_test_dt = dt.predict_proba(test_df[FEATURES])
y_test_dt = test_df['G/S']
f1_score(y_test_dt , y_pred_test_dt), roc_auc_score(y_test_dt, y_pred_score_test_dt[:, 1]), accuracy_score(y_test_dt, y_pred_test_dt)

(0.42966751918158563, 0.5, 0.2736156351791531)